In [ ]:
import tables_io
import numpy as np
import matplotlib.pyplot as plt
import pyccl as ccl
import qp
from rail.core.stage import RailStage
from rail.raruma import utility_functions as raruma_util
from rail.raruma import plotting_functions as raruma_plot
from rail.raruma import admixture_functions as raruma_admix
from rail.raruma import wrapper_classes as raruma_wrap
from scipy.stats import sigmaclip
from astropy.stats import biweight_location, biweight_scale


In [ ]:
import os

# pz_dir = '/global/cfs/cdirs/lsst/groups/PZ/DP1'  
# if that fails you can use this
# pz_dir = '/global/u2/e/echarles/dx'
#pz_dir = '/Users/echarles/pz'

# use user home directory
#pz_dir = os.path.expanduser('~')

# use user home directory plus additional folder
pz_dir = os.path.expanduser('~') + '/pz'
#pz_dir = os.path.expanduser('~') + '/dev'

print("pz_dir:",pz_dir)

In [ ]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

In [ ]:
train = tables_io.read(f"{pz_dir}/data/train/dp1_matched_v4_train.hdf5")
test = tables_io.read(f"{pz_dir}/data/test/dp1_matched_v4_test.hdf5")
#d.keys()
#train = tables_io.sliceObj(d, slice(0, -1, 10))
#test = tables_io.sliceObj(d, slice(1, -1, 10))

model_dir = "/projects/dp1_v4/data/gold_baseline"
# uncomment if model files are located alongside data folder
model_file_knn = pz_dir + model_dir + '/model_inform_knn.pkl'
model_file_bpz = pz_dir + model_dir + '/model_inform_bpz.pkl'
model_file_fzb = pz_dir + model_dir + '/model_inform_fzboost.pkl'
model_file_gpz = pz_dir + model_dir + '/model_inform_gpz.pkl'


In [ ]:
from rail.estimation.algos.k_nearneigh import KNearNeighInformer, KNearNeighEstimator
from rail.utils.catalog_utils import RubinCatalogConfig

RubinCatalogConfig.apply("com_cam")
#band_names = raruma_util.make_band_names('LSST_obs_{band}', 'ugrizy')
#error_band_names = raruma_util.make_band_names('LSST_obs_{band}_err', 'ugrizy')

In [ ]:
knn_inform = KNearNeighInformer.make_stage(nondetect_val=np.nan)


In [ ]:
KNearNeighInformer.config_options.keys()

In [ ]:
def smear_data(input_dict, noise_levels):
    # pull out the columns we want
    # smear the data
    # write the columsn back to the dict

In [ ]:
_ = plt.hist(np.log(train['redshift'].clip(1e-3, 3)), bins=100)

In [ ]:
noisy_mags = raruma_admix.gaussian_noise(train_features, noise_levels=noise*np.ones(6)).T

In [ ]:
model = knn_inform.inform(train)

In [ ]:
knn_esimate = KNearNeighEstimator.make_stage(model=model)

In [ ]:
estimate = knn_esimate.estimate(test)

In [ ]:
estimate.data.ancil

In [ ]:
_ = plt.hist2d(
     np.squeeze(estimate.data.ancil['zmode']),
    estimate.data.ancil['redshift'],
    bins=(np.linspace(0.,3, 101), np.linspace(0.,3, 101)), norm='log',
)
_ = plt.xlabel("z_true")
_ = plt.ylabel("z_est")